## Linear Regression

Lets talk about the dataset, I have downloaded [Used Car Price Dataset](https://www.kaggle.com/datasets/rishabhkarn/used-car-dataset) from Kaggle website. The dataset contains `13` feature/independent variables($x_i$) and a target/dependant variable($y$), from those 13 feature/independant variables, I will be using 5 feature/independent variables they are `kms_driven` ,`mileage(kmpl)`, `engine(cc)`, `max_power(bhp)` and `torque(Nm). ` and `price(in lakhs)` as the target variable.<br><br>


kms_driven   | mileage(kmpl)| engine(cc) | max_power(bhp) | torque(Nm) | price(in lakhs)
-------------|-------------|------------|----------------|------------|-------------
 56000       |   7.81      |2996        |	   2996        |     333    |       63.75
 30615       |  17.4      |    999     |     999        |     9863   |       8.99
24000        |  20.68     |    1995    |     1995       |      188   |       23.75
18378        |  16.5      |    1353    |     1353       |     13808  |       13.56


<br>
<br>
Here $x$'s are four-dimensional vector in $\mathbb{R}^5$. For instance, $x_1^{(i)}$ is the `kms_driven`, $x_2^{(i)}$ is the `mileage(kmpl)`, $x_3^{(i)}$ is the `engine(cc)`, $x_4^{(i)}$ is the `max_power(bhp)`, and $x_4^{(i)}$ is the `torque(Nm)` of the $i$-th house in the training set


In [330]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [332]:
cd /content/drive/MyDrive/files

/content/drive/MyDrive/files


In [333]:
ls

imdb_data.csv  Used_Car_Dataset.csv  yelp.csv


In [334]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 9.0)

In [338]:
df = pd.read_csv("Used_Car_Dataset.csv")

In [370]:
# drop all other columsn except mileage(kmpl), engine(cc), max_power(bhp), torque(Nm), price(in lakhs)
# sort the dataframe based on the price(in lakhs) column and only 100 samples are choosen
df = df[:10]
df = df[['kms_driven','mileage(kmpl)', 'engine(cc)', 'max_power(bhp)', 'torque(Nm)', 'price(in lakhs)']]
df['kms_driven'] = df['kms_driven']/1000
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kms_driven       10 non-null     float64
 1   mileage(kmpl)    10 non-null     float64
 2   engine(cc)       10 non-null     float64
 3   max_power(bhp)   10 non-null     float64
 4   torque(Nm)       10 non-null     float64
 5   price(in lakhs)  10 non-null     float64
dtypes: float64(6)
memory usage: 608.0 bytes


In [371]:
# to replicate the values across, lets set the random seed
np.random.seed(seed=123456)
X = df[['kms_driven','mileage(kmpl)', 'engine(cc)', 'max_power(bhp)', 'torque(Nm)']].to_numpy()/1000
y = df[['price(in lakhs)']].to_numpy()
#We then need to add a feature of “1” concatenating it with the dataset we already have and also add q to the vector m.
X = np.concatenate([X , np.ones((X.shape[0],1))], axis = 1)

To perform supervised learning, we will represent $y$ as  the functions/hypothese $h$ or as a linear function of $x$.

$$h_{\theta}(x) = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_3x_3 + \theta_4x_4 + \theta_5x_5$$

Here, the $\theta_i$'s are the parameters (also called wights) parameterizing the space of linear functions mapping $\mathcal{X}$ $\to$ $\mathcal{Y}$. For simplifying the notation, we drop $\theta$ and use $x_0=1$(intercept term).
$$h(x) = \sum_{i=0}^d\theta_ix_i = \theta^Tx$$

on the right-hand side both $\theta$ and $x$ are both vectors and d is the no. of feature/input/independent variables(not counting $x_0$).



Now, given a training set, how do we pick, or learn, the parameters $\theta$. One reasonable method is to make $h(x)$ close to $y$, atleast for the training examples we have. To formalize this, we will define a function that measures, for each value of the $\theta$'s, how close the $h(x^{(i)})$'s are to the correspoding $y^{(i)}$'s. We define the **cost function**:
$$J(\theta) = \frac{1}{2}\sum_{i=1}^n(h_{\theta}(x^{(i)})-y^{(i)})^2.$$



In [391]:
# contains the theta random theta values from θ1, θ2, θ3, θ4, θ5
θ_1_5 = np.random.rand(5,1)

# creating the θ0(theta zero or intecept)
θ0 = np.random.rand(1)

# complete hθ
hθ = np.concatenate([θ_1_5,q.reshape(1,-1)],axis = 0)

In [373]:
print(hθ.shape)
print(hθ)


(6, 1)
[[0.12696983]
 [0.96671784]
 [0.26047601]
 [0.89723652]
 [0.37674972]
 [0.19452593]]


# LMS algorithm


We want to choose $\theta$ so as to minimize $J(\theta)$. To do so, let's use a search algorithm that starts with some "initial guess" for $\theta$, and that repeatedly changes $\theta$ to make $J(\theta)$ smaller, until hopefully we converge to a value of $\theta$ that minimizes $J(\theta)$. Specifically, lets consider the **gradient descent** algorithm, which starts with some initial $\theta$, and repeatedly performs the update:
$$\theta_j:=\theta_j - \alpha\frac{\partial}{\partial\theta_j}J(\theta). $$
(This update is simultaneously performed for all values of $j$= 0,...,d.)


Here, $\alpha$ is called the **learning rate**. This is a very natural algorithm that repeatedly takes a step in the direction of steepest descrease of $J$.





In [374]:
# we have defined the cost fuction jθ, which is modelled after the mean square error
def mean_squared_error(X, y, hθ):
    hx = X @ hθ
    jθ = (1/2) * np.sum(((hx - y)**2), axis = 0)/len(X)
    print(jθ, "mse")
    return jθ

In order to implement this algorithm, we have to work out what is the partial derivative term on the right hand side. Lets first work it out for the case of it we have only one training example $(x, y)$, so that we can neglect the sum in the definition of $\textit{J}$. We have:


$$ \frac{\partial}{\partial\theta_j}J(\theta) = \frac{\partial}{\partial\theta_j}\frac{1}{2}(h_\theta(x) - y)^2$$
$$ = 2.\frac{1}{2}(h_\theta(x)-y).\frac{\partial}{\partial\theta_j}(h_\theta(x)-y))$$
$$=(h_\theta(x)-y).\frac{\partial}{\partial\theta_j}(\sum_{i=0}^d\theta_ix_i-y)$$
$$=(h_\theta(x)-y)x_j$$
for a single training example, this gives the update rule:
$$\theta_j:= \theta_j +\alpha(y^{(i)}-h_\theta(x^{(i)}))x_j^{(i)}. $$

This rule is called the **LMS** update rule (LMS stands for "least mean squares"), and is also known as the **Widrow-Hoff** learning rule. This rule has several properties that seem natural and intuitive. For instance, the magnitude of the update is proportial to the error term $(y^{(i)}-h_\theta(x^{(i)}))$; thus for instance, if we are encountering a training example on which our prediction nearly matches the actual value of $y^{(i)}$, then we find that there is little need to change the parameters; in contrast, a larger change to the parameters will be made if our prediction $h_\theta(x^(i)$ has a large error (i.e, if it is very far from $y^{(i)}$).

We'd derived the LMS rule for when there was only a single training example. There are two ways to modify this method for a training set of more than one example. The first is replace it with the following algorithm:

repeat until convergence $ \{ $
$$\theta_j := \theta_j + \alpha\sum_{i=i}^n(y^{(i)} - h_\theta(x^{(i)}))x_j^{(i)}, \text{(for every $j$)  (1.1)} $$
$ \}$

In [375]:
def partial_derivative(X_batch, y_batch, hθ):
    y_pred = X_batch @ hθ
    n = len(X_batch)

    df_dhθ = -(2/n) * (X_batch.T @ (y_batch - y_pred))
    df_dhθ = df_dhθ.reshape(len(df_dhθ), -1)
    return df_dhθ

In [376]:
def training_batch(X, y, batch_size, lr, epochs, hθ):

    for epoch in range(epochs):

        # random initial statistics
        # if epoch == 0:
        #     hθ = np.random.randint(low = 1, high = 20,size = (X.shape[1], 1))

        # shuffle X and y using same permutation
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)

        X = X[indices]
        y = y[indices]

        # store cumulative derivative
        cumulative_derivative = np.zeros((X.shape[1], 1))

        for batch in range(len(X)//batch_size):
            start = batch*batch_size
            stop = (batch*batch_size) + batch_size

            X_batch = X[start:stop]
            y_batch = y[start:stop]
            parti = partial_derivative(X_batch, y_batch, hθ)
            cumulative_derivative = cumulative_derivative + parti

            #updating rule
            hθ = hθ - (lr * cumulative_derivative)
        #print(f"m_stat: {m_stat}")
        print(f"epoch: {epoch} ----> MSE: {mean_squared_error(X, y, hθ)}")
    return hθ


In [389]:
batch_size = 5
lr = 0.001
epochs = 100

hθ = training_batch(X,y, batch_size,lr,epochs, hθ)
hθ

[202.65497215] mse
epoch: 0 ----> MSE: [202.65497215]
[189.21275634] mse
epoch: 1 ----> MSE: [189.21275634]
[184.58716944] mse
epoch: 2 ----> MSE: [184.58716944]
[180.11673439] mse
epoch: 3 ----> MSE: [180.11673439]
[175.88641771] mse
epoch: 4 ----> MSE: [175.88641771]
[169.56503536] mse
epoch: 5 ----> MSE: [169.56503536]
[163.03116902] mse
epoch: 6 ----> MSE: [163.03116902]
[159.94591485] mse
epoch: 7 ----> MSE: [159.94591485]
[156.35049897] mse
epoch: 8 ----> MSE: [156.35049897]
[153.28434207] mse
epoch: 9 ----> MSE: [153.28434207]
[149.94844463] mse
epoch: 10 ----> MSE: [149.94844463]
[144.53568449] mse
epoch: 11 ----> MSE: [144.53568449]
[139.85232592] mse
epoch: 12 ----> MSE: [139.85232592]
[135.41322441] mse
epoch: 13 ----> MSE: [135.41322441]
[133.12959572] mse
epoch: 14 ----> MSE: [133.12959572]
[128.86541199] mse
epoch: 15 ----> MSE: [128.86541199]
[126.44854131] mse
epoch: 16 ----> MSE: [126.44854131]
[124.04623995] mse
epoch: 17 ----> MSE: [124.04623995]
[120.08567146] mse
e

array([[ 0.19711947],
       [ 0.71957322],
       [ 6.07595809],
       [ 6.40026392],
       [-0.06673339],
       [ 1.00791242]])

By grouping the updates of the coordinates into an update of the vector $\theta$, we can rewrite update (1.1) in a slightly more succing way:
$$ \theta := \theta + \alpha\sum_{i=1}^n(y^{(i)} - h_\theta(x^{(i)}))x^{(i)}$$
This is simply gradient descent on the original cost funtion $\textit{J}$. This method looks ar every example in the entire training set on every step, and is called **batch gradient descent**. Note that, while gradient descent can be susceptible to local minima in general, the optimization problem we have posed here for linear regression has only one global, and no other local, optima; thus gradient descent always converges (assumming the learning rate $\alpha$ is not too large) to the global minimum. Indeed, $\textit{J}$ is a convex quadratic funtion. Here is an examlpe of gradient descent as it run to minimize a quadratic function.

<figure>
  <img src="https://github.com/0shankart/MachineLearningNotes/blob/main/TraditionalML/images/batch_gradient_descent_global_minimum.png?raw=true" width="40%"/>
  <figcaption>Ref: https://cs229.stanford.edu/notes2021fall/cs229-notes1.pdf</figcaption>
</figure>
 <br>
 <br>The ellipses shown above are the countours of a quadratiic function. Also shown is the trajectory inspired by gradient descent, which was initialized for $θ$ at (0.12696983, 0.96671784, 0.26047601, 0.89723652, 0.37674972, 0.19452593). The $x$'s in the figure (joined by straight lines) mark the successive values of $\theta$ that gradient descent might went through and found  (0.19711947, 0.71957322, 6.07595809, 6.40026392, -0.06673339, 1.00791242)




There is an alternative to batch gradient descent that also works very well. Consider the folling algorithm:<br><br>$\text{Loop }  \{ $

 $ \text{ for $i$ = 1 to n, } \{ $
 $$\theta^j := \theta_j + \alpha(y^{(i)} - h_\theta(x^{(i)}))x_j^{(i)}, \text{ (for every $j$)  (1.2) }$$
 $\}$

 $\}$
<br>
By grouping the updates of the coordinates into an update of the vector $\theta$, we can rewrite update (1.2) in a slightly more succing way:
$$\theta := \theta + \alpha(y^{(i)} - h_\theta(x^{(i)}))x^{(i)}$$
<br>
In this algorithm, we repeatedly run through the training set, and each time we encounter a training example, we update the parameters according to the gradient of the error with respect to that single training examlpe only. This algorithm is called **stochastic gradient descent** (also **incremental gradient descent**). Whereas batch gradient descent has to scan through the entire training set before taking a single step$-$a cpst;u p[eratopm of n is large$-$stochastic gradient descent can start making progress right away and continues to ,make progress with each example it looks at. Often, stochastic gradient descent gets $\theta$ "close to the minimum much faster than batch gradient descent. (Note however that it may never "converge" to the minimum, and the parameters $\theta$ will keep oscillating around the minimum of the $J(\theta)$; but in practise most of the values near the minimum will be reasonably good approximations to the true minimum. By slowly letting the learning rate $\alpha$ decrease to zero as the algorithm runs, it is also possible to ensure that the parameters will converge to the global minimum rather than merely oscillate around the minimum.) For these reasons, particularly when the training set is large, stochastic gradient descent is often preferred voer batch gradient descent.

